<a href="https://colab.research.google.com/github/StevenHSKim/tensorflow_study/blob/main/%ED%95%99%EC%8A%B5_%ED%9B%84_%EB%AA%A8%EB%8D%B8%EC%9D%84_%ED%8C%8C%EC%9D%BC%EB%A1%9C_%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#학습 후 모델을 파일로 저장하기

학습을 끝낸 모델을 파일로 저장하면,

파일을 이동하기도 좋고 학습을 잠깐 중지하고 다음날 다시 시작할 수도 있고 아니면 실험 후에 best 결과만 따로 저장할 수도 있습니다.

특히 AI기반 웹서비스를 만들고 싶으면 필요한게 바로 이겁니다.



예를 들면 사용자가 업로드한 사진을 개인지 고양이인지 알려주는 웹서비스를 만든다고 칩시다.

그럼 누가 사진을 업로드하면 사진을 파이썬으로 해체해서 숫자로 만든 뒤에

모델.predict(사진데이터) 이렇게 해보고 출력 결과를 유저에게 알려주면 되겠죠?

모델.predict() 할 때 필요한게 학습이 완료된 모델인데 이건 파일로 저장해놨다면 쉽게 불러올 수 있는 것이고요.



또는 자바스크립트 잘하시면 tensorflow.js 라는 라이브러리도 있어서 브라우저에서 바로 predict를 해볼 수도 있습니다.





저장할 수 있는 방법은 두개입니다.

1. model 전체 저장하기는 모델의 레이어, loss함수, optimizer, 가중치 (w값)들이 저장됩니다.

2. checkpoint 저장하기는 모델의 가중치 (w값)만 저장됩니다.

둘 중 편한거 쓰면 되는데 checkpoint는 epoch 중간중간 저장도 됩니다. 약간 더 다양한 기능을 개발할 수 있습니다.

<저장되는 항목>
1. layer 설정
2. loss 함수 종류
3. optimizer 종류
4. 훈련 후의 가중치(w) 값

In [1]:
# pj2_이미지 학습과 CNN에서 fashion_mnist 데이터셋 분류 모델 가져옴

import tensorflow as tf
import numpy as np

(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

trainX = trainX / 255.0
testX = testX / 255.0

trainX = trainX.reshape( (trainX.shape[0], 28,28,1) )
testX = testX.reshape( (testX.shape[0], 28,28,1) )

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # 주의 model.summary() 사용하려면 항상 input_shape 써줄 것
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=3)

4422102/4422102 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
1875/1875 [==============================] - 9s 4ms/step - loss: 0.4953 - acc: 0.8232 - val_loss: 0.4173 - val_acc: 0.8515
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3735 - 

#방법 1: 모델 전체를 저장하기

In [2]:
# # 현재 "model" 안에 모든 정보가 업데이트되어 있기 때문에, model.save('경로')로 저장 가능

# model.save('새폴더/model1') # '새폴더'라는 폴더를 생성해서 안에 'model1'이라는 파일을 생성하여 저장하겠다

In [6]:
# ### 모델 불러오기 ###
# 불러온_모델 = tf.keras.models.load_model('새폴더/model1')

# ### 진짜 불러왔는지 확인 ###
# 불러온_모델.summary() # 위의 summary와 동일한 것을 볼 수 있음
# 불러온_모델.evaluate(testX, testY) # .evaluate: 새로운 데이터로 모델 평가하는 함수였음

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.3779 - acc: 0.8636


[0.3779299557209015, 0.8636000156402588]

#방법 2: w값만 별도로 저장/로드하기

In [7]:
# 이 방식을 사용하면 epoch 중간 중간에 checkpoint 저장이 가능해짐

콜백함수 = tf.keras.callbacks.ModelCheckpoint(  # epoch 중간에 w가 저장되는 곳
    filepath='체크포인트/mnist', # 체크포인트라는 폴더를 만들어서, 그 안에 mnist라는 파일을 만들고 여기에 가중치를 저장하고 싶어요 (만약 error나면 직접 생성하기)
                              # 위 방법은 '체크포인트/mnist'에 가중치가 덮어씌워지는 방식으로 진행(파일이 하나만 존재).
                              # 만약 epoch마다 별도 저장하고 싶으면 '체크포인트/mnist{epoch}'로 가능

    # 이렇게 하면: validation accuracy가 최대가 되는 checkpoint만 저장할 수도 있음
    # monitor='val_acc',
    # mode='max',

    save_weights_only=True,
    save_freq='epoch'
)

# 콜백함수를 넣어서 학습하면 checkpoint 저장 가능
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=3, callbacks=[콜백함수]) # callback 여부는 model.fit에서 설정

Epoch 1/3
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3135 - acc: 0.8860 - val_loss: 0.3781 - val_acc: 0.8674
Epoch 2/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2943 - acc: 0.8908 - val_loss: 0.3724 - val_acc: 0.8688
Epoch 3/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2795 - acc: 0.8961 - val_loss: 0.3414 - val_acc: 0.8820


#Checkpoint 불러오기
방법2는 모델 전체가 아니라 w값만 저장했기 때문에, 아래처럼 모델의 선언이 필요함

In [8]:
# model2를 새로 선언했다고 하자
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model2.summary()

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

# 아래의 함수로 저장된 가중치(체크포인트) 불러올 수 있음
model2.load_weights('체크포인트/mnist')

# model의 가중치가 실제로 적용이 되어서, model2를 evaluate 했을 때 똑같은 accuracy가 나오는지 확인
model2.evaluate(testX, testY)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.3414 - acc: 0.8820


[0.341404527425766, 0.8820000290870667]